In [ ]:
import magma as m
from loam.boards.icestick import IceStick

import mantle lattice ice40


![](./images/counter_diagram.jpg)

We will use the `DefineCircuit` pattern to define a counter module.

`DefineCircuit` takes as a firt argument the name of the circuit, and then an even number of arguments of the form `port1_name, port1_type, port2_name, port2_type, ...`. By using the python `*` operator, we can programmatically construct a list of argument names and types.

For this example we compose a list literal with the output of the `ClockInterface` function which produces a standard set of inputs for a clock, clock enable, and reset. In this case, we only include a clock (by default) and an optional clock enable `has_ce=has_ce`.

We instance an n-bit `Adders` and `Register` from the `mantle` library. We wire them up based on our counter diagram above. Finally, we call `m.wireclock` to automatically wire up the clock inputs to the `Register` instance.

In [ ]:
from mantle import Add, Register


def DefineCounter(n, has_ce=False):

    name = "Counter{}{}".format(n, has_ce)

    args = ["O", m.Out(m.Bits(n)), "COUT", m.Out(m.Bit)] + m.ClockInterface(has_ce=has_ce)
    Counter = m.DefineCircuit(name, *args)

    add = Add(n, cin=False, cout=True)
    reg = Register(n, has_ce=has_ce)

    m.wire( reg.O, add.I0 )
    m.wire( m.bits(1, n), add.I1 )

    m.wire(add.O, reg.I)

    m.wire( reg.O, Counter.O )

    m.wire( add.COUT, Counter.COUT )

    m.wireclock(Counter, reg)
    m.wiredefaultclock(Counter, reg)

    m.EndDefine()
    print(repr(Counter))
    return Counter

We can inspect the generated verilog

In [ ]:
from magma.backend.verilog import compile as compile_verilog
print(compile_verilog(DefineCounter(4)))

To test our counter circuit, we will instance two versions. A 5-bit counter will be wired up to the icestick leds. Because the icestick clock runs too fast to see without a high-speed camera, we will wire up the carry out of a 23-bit counter to the clock enable of our 5-bit counter. This causes the 5-bit counter to advance slower (the clock enable will be asserted True for 1 cycle every time the 23-bit counter reaches it's maximum value).

In [ ]:
icestick = IceStick()

icestick.Clock.on()
icestick.D1.on()
icestick.D2.on()
icestick.D3.on()
icestick.D4.on()
icestick.D5.on()

main = icestick.main()

counter4 = DefineCounter(5, has_ce=True)()
counter23 = DefineCounter(23)()
m.wire(counter23.COUT, counter4.CE)
m.wire(counter4.O, m.bits([main.D1, main.D2, main.D3, main.D4, main.D5]))
print(repr(main))

Compile and flash our design onto the icestick. The leds should blink in a 5-bit counter pattern. Change the size of the 23-bit control counter or replace it with a different circuit to experiment with the counting pattern.

In [ ]:
m.compile("build/ice_counter", main)

In [ ]:
%%bash
cd build
yosys -q -p 'synth_ice40 -top main -blif ice_counter.blif' ice_counter.v
arachne-pnr -q -d 1k -o ice_counter.txt -p ice_counter.pcf ice_counter.blif
icepack ice_counter.txt ice_counter.bin

iceprog ice_counter.bin